#Data import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

# Import the CSV file containing the data.
# We assume the data file is named 'airfoil_data.csv' and is in the same directory.
# Adjust the path if necessary.
try:
    df = pd.read_csv('airfoil_data.csv')
    print("Data file loaded successfully.")
except FileNotFoundError:
    print("Error: 'airfoil_data.csv' not found.")
    print("Please make sure the file is in the correct directory.")
    sys.exit() # Exit the script if the data file is not found.


# Select 3 random entries from the airfoil database.
# sample(n=3) draws 3 random rows. random_state ensures reproducibility.
random_airfoils = df.sample(n=3, random_state=42)

# Helper function to parse coordinate strings.
def parse_coords(coord_string):
    """Converts a space-separated string of coordinates into a list of floats."""
    if isinstance(coord_string, str):
        return [float(c) for c in coord_string.strip().split()]
    return []

# Create a table-like visualization using Matplotlib.
# Create a figure with 3 rows and 2 columns.
# gridspec_kw allows setting different widths for the columns.
fig, axes = plt.subplots(3, 2, figsize=(10, 9), 
                         gridspec_kw={'width_ratios': [1, 1.5]})

fig.suptitle('Random Airfoil Profiles from the Dataset', fontsize=16)

# Iterate through the sampled airfoils and plot axes.
for i, (index, row) in enumerate(random_airfoils.iterrows()):
    # --- Left column: Textual information ---
    ax_text = axes[i, 0]
    ax_text.axis('off') # Turn off the axis for the text cell.

    # Collect data for display.
    name = row['name']
    angle = row['angle']
    reynolds = row['reynolds']
    cd = row['cd']
    cl = row['cl']
    cm = row['cm']
    
    # Create the formatted text string.
    info_text = (
        f"Name: {name}\n\n"
        f"Angle of Attack: {angle}°\n"
        f"Reynolds Number: {reynolds:,.0f}\n\n"
        f"Coefficients:\n"
        f"  Cd: {cd:.4f}\n"
        f"  Cl: {cl:.4f}\n"
        f"  Cm: {cm:.4f}"
    )
    
    # Display the text in the cell.
    ax_text.text(0.05, 0.5, info_text, transform=ax_text.transAxes,
                 fontsize=11, verticalalignment='center', family='monospace')

    # --- Right column: Airfoil shape plot ---
    ax_plot = axes[i, 1]

    # Parse the coordinates.
    x_coords = parse_coords(row['x_coords'])
    y_coords = parse_coords(row['y_coords'])
    
    # Plot the airfoil shape.
    ax_plot.plot(x_coords, y_coords, marker='o', markersize=3, linestyle='-', color='b')
    ax_plot.scatter([x_coords[0]], [y_coords[0]], color='red', s=40, zorder=5, label='Start/End Point') # Highlight the start point.
    
    # Configure the plot settings.
    ax_plot.set_title(f'Airfoil Shape: {name}')
    ax_plot.set_xlabel('x')
    ax_plot.set_ylabel('y')
    ax_plot.grid(True, linestyle='--', alpha=0.6)
    # set_aspect('equal') is crucial to maintain the airfoil's proportions.
    ax_plot.set_aspect('equal', adjustable='box')
    ax_plot.legend()

# Adjust the layout to prevent elements from overlapping.
# rect leaves space for the main title.
plt.tight_layout(rect=[0, 0, 1, 0.96]) 
plt.show()

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class AirfoilLSTMRegressor(nn.Module):
    def __init__(self, point_dim=2, lstm_hidden_dim=128, num_lstm_layers=2, 
                 fc_hidden_dim=64, output_dim=3, other_features_dim=2, dropout_prob=0.2):
        super(AirfoilLSTMRegressor, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=point_dim,
            hidden_size=lstm_hidden_dim,
            num_layers=num_lstm_layers,
            batch_first=True,
            bidirectional=False
        )

        self.fc1 = nn.Linear(lstm_hidden_dim + other_features_dim, fc_hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(fc_hidden_dim, output_dim)

    def forward(self, sequence_data, sequence_lengths, other_features):
        packed_input = pack_padded_sequence(
            sequence_data, 
            sequence_lengths.cpu(),
            batch_first=True, 
            enforce_sorted=False
        )
        
        packed_lstm_out, (h_n, _) = self.lstm(packed_input)
        shape_representation = h_n[-1,:,:]
        combined_features = torch.cat((shape_representation, other_features), dim=1)
        x = self.fc1(combined_features)
        x = self.relu(x)
        x = self.dropout(x)
        predictions = self.fc2(x)
        
        return predictions


#Przykładowe użycie

In [ ]:

point_dim = 2                 # (x, y)
lstm_hidden_dim = 128
num_lstm_layers = 2
fc_hidden_dim = 64
output_dim = 3                # CL, CD, CM
other_features_dim = 2        # Re, AoA

model = AirfoilLSTMRegressor(point_dim, lstm_hidden_dim, num_lstm_layers, 
                             fc_hidden_dim, output_dim, other_features_dim)
